In [1]:
%load_ext autoreload


In [2]:
__author__ = 'biherrera'
__email__ = 'biherrera@uc.cl'

%autoreload

import sys
import os

# The following line adds the directory to the path in order to cross-reference
# files in the repo
sys.path.append(os.getcwd()[:-6])
print("Relative path: ", os.getcwd()[:-6])
import dolfin
from fenics import *
from src.model import PerfusionGasExchangeModel
from src.params import params

Relative path:  /home/bnherrerac/alveolar-perfusion-transport-modeling


SyntaxError: invalid syntax (model.py, line 434)

In [ ]:
# print("Imported src files")
# print("Starting...")
# folder = "rve_25_smooth"
# path = os.path.join("../local-results-data", folder)
# print(path)
# model = PerfusionGasExchangeModel(folder_path=path, params=params)
# print("Model initialised")
# model.import_mesh(
#     os.path.join("../raw-data/rve_25_smooth", "tetra_mesh_25_smooth.xdmf"), type="xdmf", 
#     periodic=False, max_dims = [24.630131, 24.657419, 24.623438], min_dims = [0.337988, 0.372257, 0.350474], tol=0.7
# )
# print("Mesh imported")
# model.mesh = dolfin.refine(model.mesh)
# print("Mesh refined")
# print("Starting (P) simulation")
# model.sim_p(save=True, meshtype="tkd")
# print("(P) simulation done")
# print("Starting (T) simulation")
# x = model.sim_t(hb=False, save=True)
# print("Finished (linear) guess generation")
# solution = model.sim_t(hb=True, save=True, guess=x)
# print("Done")

Relative path:  /home/bnherrerac/alveolar-perfusion-transport-modeling
Imported src files
Starting...
../local-results-data/rve_25_smooth
Model initialised
Mesh imported
Coordinates have shape  (4575, 3)
Mesh imported
Mesh refined
Starting (P) simulation
began sim_p
Instancing boundaries
gamma_in.dir_min =  0.337988
gamma_in.dir_max =  24.630131
gamma_out.dir_min =  0.337988
gamma_out.dir_max =  24.630131
gamma_air.dir_min_y =  0.372257
gamma_air.dir_max_y =  24.657419
gamma_air.dir_min_z =  0.350474
gamma_air.dir_max_z =  24.623438
boundaries instanced
P problem solved
u problem solved
Saving solutions
saved
(P) simulation done
Starting (T) simulation
Finished (linear) guess generation


In [19]:
list_linear_solver_methods()
list_krylov_solver_preconditioners()

In [5]:
print(dolfin.__version__)

2019.2.0.dev0
